In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix


In [2]:
class SpotifyDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class SpotifyRankPredictor(nn.Module):
    def __init__(self, num_categories):
        super(SpotifyRankPredictor, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(8, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.2),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.BatchNorm1d(16),
            nn.Linear(16, num_categories)  # 출력층이 1개에서 5개(카테고리 수)로 변경
        )
        self.softmax = nn.Softmax(dim=1)  # Softmax 활성화 함수 추가하여 각 카테고리의 확률 출력

    def forward(self, x):
        x = self.layers(x)
        return self.softmax(x)

In [3]:
def preprocess_data(df):
    X = df[['Danceability', 'Energy', 'Loudness', 'Speechiness',
            'Acousticness', 'Liveness', 'Tempo', 'Duration (ms)']].values

    feature_scaler = MinMaxScaler()
    X_scaled = feature_scaler.fit_transform(X)

    ranks = df['Highest Charting Position'].values

    # 다섯개의 카테고리로 나눔
    def rank_to_category(rank):
        if rank <= 10:
            return 0  # Top 10
        elif rank <= 30:
            return 1  # Top 11-30
        elif rank <= 50:
            return 2  # Top 31-50
        elif rank <= 100:
            return 3  # Top 51-100
        else:
            return 4  # Below 100

    
    y_categorical = np.array([rank_to_category(rank) for rank in ranks])

    # One-hot encoding 적용
    num_categories = 5
    y_encoded = np.eye(num_categories)[y_categorical]

    return X_scaled, y_encoded, num_categories

In [4]:
def train_model(model, train_loader, val_loader, criterion, optimizer,
                num_epochs=100, patience=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    best_val_loss = float('inf')
    patience_counter = 0
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)  # Classification 문제로 바뀌었기 때문에 이 부분 수정. 기존에는 y_batch(-1,1)여서 연속적인 값 예측시 사용하는 것
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)
        train_losses.append(train_loss)

        model.eval()
        val_loss = 0
        correct = 0
        total = 0

        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                _, actual = torch.max(y_batch.data, 1)
                total += y_batch.size(0)
                correct += (predicted == actual).sum().item()

        val_loss /= len(val_loader)
        val_losses.append(val_loss)
        accuracy = 100 * correct / total

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, '
              f'Val Loss: {val_loss:.4f}, Val Accuracy: {accuracy:.2f}%')

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print('Early stopping triggered')
                break

    return train_losses, val_losses

In [5]:
def compute_class_weights(y):
    # y는 one-hot encoded 형태이므로 실제 클래스로 변환
    y_classes = np.argmax(y, axis=1)

    # 각 클래스별 샘플 수 계산
    class_counts = np.bincount(y_classes)

    # 가중치 계산: 전체 샘플 수 / (클래스 수 * 각 클래스의 샘플 수)
    weights = len(y) / (len(class_counts) * class_counts)

    return torch.FloatTensor(weights)

In [6]:
def main():
    df = pd.read_csv('spotify_dataset.csv')
    X_scaled, y_encoded, num_categories = preprocess_data(df)

    X_train, X_temp, y_train, y_temp = train_test_split(
        X_scaled, y_encoded, test_size=0.2, random_state=42
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=42
    )

    train_dataset = SpotifyDataset(X_train, y_train)
    val_dataset = SpotifyDataset(X_val, y_val)
    test_dataset = SpotifyDataset(X_test, y_test)

    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    class_weights = compute_class_weights(y_train)
    print("Class weights:", class_weights)

    model = SpotifyRankPredictor(num_categories)
    criterion = nn.CrossEntropyLoss(weight=class_weights)  #CrossEntropy에서 가중치 적용
    optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.0001)

    train_losses, val_losses = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        num_epochs=100,
        patience=10
    )

    model.load_state_dict(torch.load('best_model.pth'))
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs.data, 1)
            _, actual = torch.max(y_batch.data, 1)
            total += y_batch.size(0)
            correct += (predicted == actual).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

    y_pred = []
    y_true = []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs.data, 1)
            _, actual = torch.max(y_batch.data, 1)
            y_pred.extend(predicted.numpy())
            y_true.extend(actual.numpy())

    conf_matrix = confusion_matrix(y_true, y_pred)
    print("\nConfusion Matrix:")
    print(conf_matrix)

if __name__ == "__main__":
    main()

Class weights: tensor([1.6298, 1.3223, 1.8540, 0.7718, 0.5571])
Epoch [1/100], Train Loss: 1.6135, Val Loss: 1.4736, Val Accuracy: 13.24%
Epoch [2/100], Train Loss: 1.6136, Val Loss: 1.4744, Val Accuracy: 15.07%
Epoch [3/100], Train Loss: 1.6099, Val Loss: 1.4734, Val Accuracy: 15.07%
Epoch [4/100], Train Loss: 1.6099, Val Loss: 1.4735, Val Accuracy: 15.98%
Epoch [5/100], Train Loss: 1.6069, Val Loss: 1.4747, Val Accuracy: 17.35%
Epoch [6/100], Train Loss: 1.6072, Val Loss: 1.4732, Val Accuracy: 15.98%
Epoch [7/100], Train Loss: 1.6009, Val Loss: 1.4725, Val Accuracy: 15.53%
Epoch [8/100], Train Loss: 1.5996, Val Loss: 1.4728, Val Accuracy: 15.53%
Epoch [9/100], Train Loss: 1.5992, Val Loss: 1.4739, Val Accuracy: 15.98%
Epoch [10/100], Train Loss: 1.5966, Val Loss: 1.4753, Val Accuracy: 15.53%
Epoch [11/100], Train Loss: 1.5999, Val Loss: 1.4763, Val Accuracy: 15.53%
Epoch [12/100], Train Loss: 1.5985, Val Loss: 1.4773, Val Accuracy: 17.81%
Epoch [13/100], Train Loss: 1.5970, Val Loss:

C:\Users\이강민\AppData\Local\Temp\ipykernel_11216\598819680.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))
